In [2]:
import os
os.chdir('./..')

In [3]:
import data

In [4]:
import pandas as pd

In [5]:
import numpy as np

In [6]:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [7]:
import matplotlib.pyplot as plt

In [8]:
from utils.check_folder import check_folder

In [9]:
import pickle

In [10]:
from tqdm.auto import tqdm  # for notebooks
tqdm.pandas()

## Build the dataset

In [11]:
def existsIndex(df, idx):
    try:
        df.loc[idx]
        return True
    except KeyError:
        return False

In [12]:
def RepresentsInt(s):
    try: 
        int(s)
        return True
    except TypeError:
        return False
    except ValueError:
        return False

In [13]:
df = data.train_df('full').head(1000000)

/home/giovanni/miniconda3/lib/python3.7/site-packages/numpy/lib/arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [319]:
def func(x):
    y = x[x['action_type'] == 'clickout item']
    if len(y) > 0:
        clk = y.tail(1)
        head_index = x.head(1).index
        
        # considering only the past!
        x = x.loc[head_index.values[0]:clk.index.values[0]-1]
        
        impr = clk['impressions'].values[0].split('|')
        prices = list(map(int, clk['prices'].values[0].split('|')))
        sorted_prices = prices.copy()
        sorted_prices.sort()
        
        references = x['reference'].values
        
        # features
        features = {'times_impression_appeared': [], #'time_elapsed_from_last_time_impression_appeared': [], 
                    'steps_from_last_time_impression_appeared': [], 'kind_action_reference_appeared': [], 
                    'impression_position': [], 'label': [], 'price': [], 'price_position': []}
        count = 0
        for i in impr:
            indices = np.where(references == str(i))[0]
            
            features['impression_position'].append(count+1)
            features['price'].append(prices[count])
            features['price_position'].append(sorted_prices.index(prices[count]))
            if len(indices) > 0:
                row_reference = x.head(indices[-1]+1).tail(1)
                features['steps_from_last_time_impression_appeared'].append(len(x)-indices[-1])
                #features['time_elapsed_from_last_time_impression_appeared'].append(int(clk['timestamp'].values[0] - row_reference['timestamp'].values[0]))
                features['kind_action_reference_appeared'].append(row_reference['action_type'].values[0])
            else:
                features['steps_from_last_time_impression_appeared'].append(0)
                #features['time_elapsed_from_last_time_impression_appeared'].append(np.inf)
                features['kind_action_reference_appeared'].append('no_action')
    
            if clk['reference'].values[0] == i:
                features['label'].append(1)
                features['times_impression_appeared'].append(len(indices))
            else:
                features['label'].append(0)
                features['times_impression_appeared'].append(len(indices))
            
            count += 1
        return pd.DataFrame(features)


In [320]:
dataset = df.groupby(['user_id', 'session_id']).progress_apply(func)
one_hot = pd.get_dummies(dataset['kind_action_reference_appeared'])
dataset = dataset.drop(['kind_action_reference_appeared'],axis = 1)
dataset = dataset.join(one_hot)

In [321]:
dataset

times_impression_appeared  \
user_id      session_id                                    
0004IOZI7CKF 0146f7cb014ba 0                           0   
                           1                           0   
                           2                           0   
                           3                           0   
                           4                           0   
                           5                           0   
                           6                           0   
                           7                           0   
                           8                           0   
                           9                           0   
                           10                          0   
                           11                          0   
                           12                          0   
                           13                          0   
                           14                          0   
                           15                          0   
                           16                          0   
                           17                          0   
                           18                          0   
                           19                          0   
                           20                          0   
                           21                          0   
                           22                          0   
                           23                          0   
                           24                          0   
0008B0X0HC39 1b703eda68d85 0                           0   
000EKQEFMW6S c53fab5092ec2 0                           0   
                           1                           0   
                           2                           0   
                           3                           0   
...                                                  ...   
ZZYRG6A6KZB7 1d6f80c36b646 0                           0   
                           1                           0   
                           2                           0   
                           3                           0   
                           4                           0   
                           5                           0   
                           6                           0   
                           7                           0   
                           8                           0   
                           9                           0   
                           10                          0   
                           11                          0   
                           12                          0   
                           13                          0   
                           14                          0   
                           15                          0   
                           16                          0   
                           17                          0   
                           18                          0   
                           19                          0   
                           20                          0   
                           21                          0   
                           22                          0   
                           23                          0   
                           24                          0   
ZZZ4CA1PZ1VU 5fad2053f6f2e 0                           0   
                           1                           0   
                           2                           0   
                           3                           0   
                           4                           0   

                               steps_from_last_time_impression_appeared  \
user_id      session_id                                                   
0004IOZI7CKF 0146f7cb014ba 0                                          0   
                         

In [322]:
X, y = dataset.iloc[:, [1,2,4,5,6,7,8,9,10]],dataset.iloc[:,3]

## Split the data

In [323]:
import math

In [324]:
X_train = X.head(math.ceil(0.8 * len(X)))
y_train = y.head(math.ceil(0.8 * len(X)))
X_test = X.tail(math.floor(0.2 * len(X)))
y_test = y.tail(math.floor(0.2 * len(X)))

In [ ]:
y_train

In [326]:
X_train

steps_from_last_time_impression_appeared  \
user_id      session_id                                                   
0004IOZI7CKF 0146f7cb014ba 0                                          0   
                           1                                          0   
                           2                                          0   
                           3                                          0   
                           4                                          0   
                           5                                          0   
                           6                                          0   
                           7                                          0   
                           8                                          0   
                           9                                          0   
                           10                                         0   
                           11                                         0   
                           12                                         0   
                           13                                         0   
                           14                                         0   
                           15                                         0   
                           16                                         0   
                           17                                         0   
                           18                                         0   
                           19                                         0   
                           20                                         0   
                           21                                         0   
                           22                                         0   
                           23                                         0   
                           24                                         0   
0008B0X0HC39 1b703eda68d85 0                                          0   
000EKQEFMW6S c53fab5092ec2 0                                          0   
                           1                                          0   
                           2                                          0   
                           3                                          0   
...                                                                 ...   
S2HB9U70T9LE 4813b196f5d2b 22                                         0   
                           23                                         0   
                           24                                         0   
S2HBRAFAIIS5 6576766145cc1 0                                          0   
                           1                                          0   
                           2                                          0   
                           3                                          0   
                           4                                          0   
                           5                                          0   
                           6                                          0   
                           7                                          0   
                           8                                          0   
                           9                                          0   
                           10                                         0   
                           11                                         0   
                           12                                         0   
                           13                                         0   
                           14                                         0   
                           15                                         0   
                           16                                         0   
                           17                           

In [397]:
X_train[X_train['steps_from_last_time_impression_appeared'] != 0]

steps_from_last_time_impression_appeared  \
user_id      session_id                                                   
002BISXP1U1Q 8cd721ffb8e03 0                                          1   
002O8H8GYKSQ ab252cd2bafd3 1                                          1   
003ZD97CWBYE f833a387fa0c9 0                                          2   
                           1                                          3   
                           2                                          1   
004STL2FQU99 24ab1f448fa9c 5                                          1   
006MM63UET1V e5d37102d5f21 1                                          1   
006OQ8Q1YFFY cbe7976eabb3f 0                                          5   
                           6                                          2   
                           7                                          1   
00941AJZU07K a337682a6f448 0                                         14   
                           1                                         16   
                           2                                         15   
                           4                                          2   
                           5                                          1   
                           6                                         12   
                           10                                        13   
00A86US7B103 8c8cab541a53b 1                                          1   
                           4                                         28   
00C7567H3C76 ea8eabf02253d 0                                          1   
00CX1JA8LW0S 2281a4607607a 0                                          3   
                           2                                          2   
                           3                                          1   
00E8MKDYI6MC 79d78f9e2523e 0                                          2   
                           1                                          1   
                           2                                          4   
00FEK9OY1TSS 176f4c54419b6 0                                          1   
00FXSTAWAMR3 4de9138cb34dd 3                                          2   
                           9                                          1   
00FXTZ1X2KHW cef1e94b198b0 0                                          1   
...                                                                 ...   
S1XZNW2O4NEE 1680a6e883da1 0                                          1   
S1Z2Y2VPB6HR 3ea49c25aba65 9                                         33   
                           16                                         1   
S1ZH18QETXX3 799a43a9048ce 0                                          1   
S1ZK6O5OWN4J 1f3ee91c4f978 3                                          1   
S20C2D5HI4Q1 85ae85a8f810a 1                                          1   
S24A3WE5Z205 c04bebd7113ff 14                                         1   
S24CT52N6T54 8158250c70fe7 2                                          1   
S24RN5BNI808 d506dc8af8cba 1                                          7   
                           2                                          3   
                           7                                         12   
                           9                                          6   
S25FA9QBSW6B 4dea3485e0e5f 0                                          1   
S25JJCJ2IQHK 44b17cf4d6afb 0                                          1   
S25NRH0TZDJD 48dc198602c20 11                                        56   
                           12                                        24   
                           14                                        11   
                           17                                         1   
                           18                                        22   
                           20                                        55   
S25RQUL05OUJ 45f12ad4fbcf5 0                            

## Train on it

In [386]:
from keras import Sequential
from keras.layers import Dense
from keras import regularizers

In [387]:
classifier = Sequential()
classifier.add(Dense(200, activation='relu', kernel_initializer='random_normal', input_dim=9, kernel_regularizer=regularizers.l2(0.001)))
classifier.add(Dense(200, activation='relu', kernel_initializer='random_normal', kernel_regularizer=regularizers.l2(0.001)))
classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))

In [388]:
classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])

In [389]:
for i in range(10):
    classifier.fit(X_train,y_train, batch_size=10, epochs=1)
    preds=classifier.predict(X_test)
    df_preds = pd.DataFrame(y_test.reset_index()).join(pd.DataFrame({'scores': list(preds[:,0])}))
    results = df_preds.groupby(['user_id', 'session_id']).progress_apply(comp_mrr)
    print('mrr is {}'.format(sum(results['score'].values)/len(results['score'].values)))

Epoch 1/1
945740/945740 [==============================] - 228s 241us/step - loss: 0.1416 - acc: 0.9594


mrr is 0.6261844517177914
Epoch 1/1
945740/945740 [==============================] - 225s 238us/step - loss: 0.1358 - acc: 0.9608


mrr is 0.6267669220431823
Epoch 1/1
945740/945740 [==============================] - 223s 236us/step - loss: 0.1355 - acc: 0.9604


mrr is 0.6179666404554026
Epoch 1/1
945740/945740 [==============================] - 225s 238us/step - loss: 0.1350 - acc: 0.9607


mrr is 0.6143193852012255
Epoch 1/1
 42670/945740 [>.............................] - ETA: 3:38 - loss: 0.1349 - acc: 0.9611

KeyboardInterrupt: 

## Compute MRR

In [ ]:
def comp_mrr(x):
    score = 0
    label = list(map(int, x['label'].values))
    scores = list(x['scores'].values)
    if 1 in label:
        count = 1
        while True:
            idx_1 = label.index(1)
            emax = max(scores)
            idx_max = scores.index(emax)
            if idx_1 == idx_max:
                score = 1/count
                break
            else:
                count += 1
                scores[idx_max] = -np.inf
        return pd.DataFrame({'score': [score]})
    else:
        return pd.DataFrame()

## Visualize the results

In [372]:
preds[preds >= 0.5] = 1
preds[preds < 0.5] = 0

In [373]:
target_names = ['clk_diff_ref', 'clk_eq_ref']
print(classification_report(y_test, preds, target_names=target_names))

              precision    recall  f1-score   support

clk_diff_ref       0.96      1.00      0.98    226069
  clk_eq_ref       0.78      0.20      0.32     10365

   micro avg       0.96      0.96      0.96    236434
   macro avg       0.87      0.60      0.65    236434
weighted avg       0.96      0.96      0.95    236434



In [374]:
accuracy = accuracy_score(y_test, preds)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 96.26%


## Dummy baseline (always say 0)

In [37]:
dummy = np.zeros(len(preds), dtype=np.int)

In [38]:
target_names = ['clk_diff_ref', 'clk_eq_ref']
print(classification_report(y_test, dummy, target_names=target_names))

              precision    recall  f1-score   support

clk_diff_ref       0.96      1.00      0.98    110364
  clk_eq_ref       0.00      0.00      0.00      5105

   micro avg       0.96      0.96      0.96    115469
   macro avg       0.48      0.50      0.49    115469
weighted avg       0.91      0.96      0.93    115469



/home/giovanni/miniconda3/lib/python3.7/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [39]:
accuracy = accuracy_score(y_test, dummy)
print("Accuracy: %.2f%%" % (accuracy * 100.0))

Accuracy: 95.58%
